In [148]:
from bs4 import BeautifulSoup
import lxml
from lxml.etree import tostring as htmlstring
import requests

from fake_useragent import FakeUserAgent

from dotenv import dotenv_values

import re
import csv
import os.path
from datetime import date, timedelta

import pandas as pd

In [72]:
# Подгрузка данных из файла окружения
config = dotenv_values(".env")

# Обращения
# config[hh_api_name]
# config[hh_api_Client_ID]
# config[hh_api_Client_Secret]

In [92]:
# Основные регулярные выражения для проекта
re_vacancy_id_hh = r'\/vacancy\/(\d+)\?'
re_vacancy_id_rabota = r'\/vacancy\/(\d+)'
re_vacancy_id_finder = r'\/vacancies\/(\d+)'
re_vacancy_id_zarplata = r'\/vacancy\/card\/id(\d+)'

# re.search(re_vacancy_id, string).group(1)

In [ ]:
class Rabota1000_Parser:
    # Класс для парсинга вакансий с ресурса Rabota1000.ru
    def __init__(self, city:str='russia'):
        self.max_page_count = 5
        self.basic_url = 'https://rabota1000.ru/russia/'
        self.vac_name_list = []
        self.vac_name_list = self.get_vac_name_into_file('vacancy_name_to_pars.txt')
        
        ua = FakeUserAgent()
        headers = {'user-agent':ua.random}

        self._pars()

    def _pars(self):
        if not os.path.exists('pars_link.csv'):
            with open('pars_link.csv', 'w', newline='', encoding='utf-8') as csv_file:
                names = ['vac_name', 'link', 'source', 'vac_id']
                file_writer = csv.DictWriter(csv_file, delimiter = ",", lineterminator="\r", fieldnames=names)
                file_writer.writeheader()

            for vac_name in self.vac_name_list:
                print(vac_name)
                try:
                    used_url = self.basic_url + vac_name + "/"
                    response = requests.get(used_url)
                    response.raise_for_status()
                    for i in range(1, self.max_page_count+1):
                        print(i, end=' ')
                        used_url = f'{self.basic_url}{vac_name}?p={i}'
                        page = requests.get(used_url)
                        soup = BeautifulSoup(page.text, 'html.parser')

                        # 20 ссылок на одной странице
                        links = [requests.get(link['href']).url for link in soup.findAll('a', attrs={'@click':'vacancyLinkClickHandler'})]
                        sources = [source.text for source in soup.findAll('span', attrs={'class':'text-sky-600'})]

                        links_to_save = [[vac_name, link, source] for link, source in zip(links, sources)]

                        with open('pars_link.csv', 'a', newline='', encoding='utf-8') as csv_file:
                            writer = csv.writer(csv_file)
                            writer.writerows(links_to_save)
                            
                except HTTPError as exc:
                    code = exc.response.status_code
                    print(code)
                print()

        links_for_processing = []
        with open('pars_link.csv', 'r', encoding='utf-8') as csv_file:
            reader = csv.reader(csv_file)
            labels = next(reader, None)
            for row in reader:
                links_for_processing.append(dict(zip(labels, row)))
        
        for item in links_for_processing:
            if item['source'] == 'hh.ru':
                item['vac_id'] = re.search(re_vacancy_id_hh, item['link']).group(1)
            elif item['source'] == 'finder.vc':
                item['vac_id'] = re.search(re_vacancy_id_finder, item['link']).group(1)
            elif item['source'] == 'zarplata.ru':
                item['vac_id'] = re.search(re_vacancy_id_zarplata, item['link']).group(1)
            else:
                item['vac_id'] = re.search(re_vacancy_id, item['link']).group(1)

        pre_resualt = []
        for link in links_for_processing:
            if link['source'] == 'hh.ru':
                pre_resualt.append(self._pars_url_hh(link['vac_id']))
            elif link['source'] == 'zarplata.ru':
                pre_resualt.append(self._pars_url_zarplata(link['vac_id']))
            elif item['source'] == 'finder.vc':
                pre_resualt.append(self._pars_url_finder(link['vac_id']))
            else:
                print(link['link'])
                pre_resualt.append(self._pars_url_other(link['vac_id']))

    def get_vac_name_into_file(self, vac_file_path:str)->list:
        vac_name_list = []
        with open(vac_file_path, mode='r', encoding="utf-8") as file_vac:
            vac_name_list = list(map(lambda x: x.lower().replace('\n', '').replace(' ','+'), file_vac.readlines()))

        return vac_name_list

    def _pars_url_hh(self, id:str)->dict:
        res = {}
        data = requests.get(f'https://api.hh.ru/vacancies/{id}').json()
        res['vac_link'] = f'https://hh.ru/vacancy/{id}' # Ссылка
        res['name'] = data['name']                      # Название
        res['city'] = data['area']['name']              # Город
        res['company'] = data['employer']['name']       # Назвнание компании публикующей вакансию
        res['experience'] = data['experience']['name']  # Опыт работы (нет замены на jun mid и sin)
        res['schedule'] = data['schedule']['name']      # Тип работы (офис/удаленка и тд)
        res['employment'] = data['employment']['name']  # График работы
        res['skills'] = data['key_skills']              # Ключевые навыки
        res['description'] = data['description']        # Полное описание (html теги не убраны)
        if data['salary'] == None: 
            res['salary'] = 'Договорная'                # Если ЗП не указано то пишем договорная
        else:
            res['salary'] = data['salary']              # Если есть то берем как есть
        res['time'] = data['published_at']              # Дата и время публикации

        return res


    def _pars_url_zarplata(self, id:str)->dict:
        res = {}
        data = requests.get(f'https://api.zarplata.ru/vacancies/{id}').json()
        res['vac_link'] = f'https://www.zarplata.ru/vacancy/card/id{id}' # Ссылка
        res['name'] = data['name']                      # Название
        res['city'] = data['area']['name']              # Город
        res['company'] = data['employer']['name']       # Назвнание компании публикующей вакансию
        res['experience'] = data['experience']['name']  # Опыт работы (нет замены на jun mid и sin)
        res['schedule'] = data['schedule']['name']      # Тип работы (офис/удаленка и тд)
        res['employment'] = data['employment']['name']  # График работы
        res['skills'] = data['key_skills']              # Ключевые навыки
        res['description'] = data['description']        # Полное описание (html теги не убраны)
        if data['salary'] == None: 
            res['salary'] = 'Договорная'                # Если ЗП не указано то пишем договорная
        else:
            res['salary'] = data['salary']              # Если есть то берем как есть
        res['time'] = data['published_at']
        
        return res


    def _pars_url_other(self, id:str)->dict:
        res = {}
        soup = BeautifulSoup(requests.get(f'https://rabota1000.ru/vacancy/{id}').text, 'html.parser')
        res['vac_link'] = f'https://rabota1000.ru/vacancy/{id}' # Ссылка
        res['name'] = soup.find('h2').text                      # Название
        res['city'] = data['area']['name']              # Город
        res['company'] = data['employer']['name']       # Назвнание компании публикующей вакансию
        res['experience'] = data['experience']['name']  # Опыт работы (нет замены на jun mid и sin)
        res['schedule'] = data['schedule']['name']      # Тип работы (офис/удаленка и тд)
        res['employment'] = data['employment']['name']  # График работы
        res['skills'] = data['key_skills']              # Ключевые навыки
        res['description'] = data['description']        # Полное описание (html теги не убраны)
        if data['salary'] == None: 
            res['salary'] = 'Договорная'                # Если ЗП не указано то пишем договорная
        else:
            res['salary'] = data['salary']              # Если есть то берем как есть
        res['time'] = data['published_at']
        
    def _pars_url_finder(self, id:str)->list:
        res = {}
        soup = BeautifulSoup(requests.get(f'https://finder.vc/vacancies/{id}').text, 'html.parser')
        dom = lxml.etree.HTML(str(soup)) 
        res['vac_link'] = f'https://finder.vc/vacancies/{id}' # Ссылка
        res['name'] = soup.find('h1', attrs={'class':'vacancy-info-header__title'}).text # Название
        res['city'] = ''              # Город (НЕТ)
        res['company'] = dom.xpath('/html/body/div[1]/div[2]/div/main/div/div/div[2]/div[1]/div/div/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/a')[0].text        # Назвнание компании публикующей вакансию
        res['experience'] = dom.xpath('/html/body/div[1]/div[2]/div/main/div/div/div[2]/div[1]/div/div/div[1]/div/div[2]/div[3]/div[1]/div[2]/div')[0].text  # Опыт работы (нет замены на jun mid и sin)
        res['schedule'] = ''     # Тип работы (офис/удаленка и тд) (НЕТ
        res['employment'] = dom.xpath('/html/body/div[1]/div[2]/div/main/div/div/div[2]/div[1]/div/div/div[1]/div/div[3]/div/div[2]/a')[0].text # График работы
        res['skills'] = [li.text for li in dom.xpath('/html/body/div[1]/div[2]/div/main/div/div/div[2]/div[1]/div/div/div[3]/div[1]/div[2]/div[1]/ul')[0]]           # Ключевые навыки
        res['description'] = ''    # Полное описание (НЕТ)
        res['salary'] = dom.xpath('/html/body/div[1]/div[2]/div/main/div/div/div[2]/div[1]/div/div/div[1]/div/div[2]/div[2]/div[2]/div')[0].text.replace(u'\xa0', '')

        if 'сегодня' in dom.xpath('/html/body/div[1]/div[2]/div/main/div/div/div[2]/div[1]/div/div/div[1]/div/div[1]')[0].text:
            res['time'] = str(date.today())
        elif 'вчера' in dom.xpath('/html/body/div[1]/div[2]/div/main/div/div/div[2]/div[1]/div/div/div[1]/div/div[1]')[0].text:
            res['time'] = str(date.today() - timedelta(days=1))
        else:
            res['time'] = str(date.today() - timedelta(days=int(re.search(r'Опубликована (\d+)', dom.xpath('/html/body/div[1]/div[2]/div/main/div/div/div[2]/div[1]/div/div/div[1]/div/div[1]')[0].text).group(1))))

        return res

    

parser = Rabota1000_Parser()

Необходимо достать в результате

- Название вакансии +
- Название компании +
- З/п +
- Формат работы +
- Тип занятости +
- Навыки (Если можно достать отдельно) +
- Описание вакансии (если есть требования или навыки - то отлично, берем их) +
- Время публикации +
- Ссылка на вакансию тоже должна быть у нас +

In [101]:
id = 103050901
print(f'https://rabota1000.ru/vacancy/{id}')
soup = BeautifulSoup(requests.get(f'https://rabota1000.ru/vacancy/{id}').text, 'html.parser')
res['vac_link'] = f'https://rabota1000.ru/vacancy/{id}' # Ссылка
res['name'] = soup.find('h2').text                      # Название
res['city'] = ''              # Город
res['company'] =        # Назвнание компании публикующей вакансию
res['experience'] =   # Опыт работы (нет замены на jun mid и sin)
res['schedule'] =       # Тип работы (офис/удаленка и тд)
res['employment'] =  # График работы
res['skills'] =             # Ключевые навыки
res['description'] =        # Полное описание (html теги не убраны)
res['salary'] = []
res['time'] = data['published_at']

print(res)

https://rabota1000.ru/vacancy/103050901
{'vac_link': 'https://rabota1000.ru/vacancy/103050901', 'name': '\n                            Младший воспитатель                        ', 'city': 'Москва', 'company': 'Рултек', 'experience': 'Нет опыта', 'schedule': 'Полный день', 'employment': 'Полная занятость', 'skills': [{'name': 'Мобильность'}, {'name': 'Пользователь ПК'}], 'description': '<p><strong>Требуется курьер </strong>на доставку документов</p> <p> </p> <p><strong>Условия:</strong></p> <p>Доставка документов</p> <p>Проезд и мобильная связь оплачивается в полном размере.</p> <p>Зарплата выплачивается регулярно, 2 раза в месяц.</p>', 'salary': {'from': 28000, 'to': 35000, 'currency': 'RUR', 'gross': False}, 'time': '2023-10-05T12:08:57+0300'}


In [149]:
id = 32381



print(res)

{'vac_link': 'https://finder.vc/vacancies/32381', 'name': 'Junior data-аналитик', 'city': '', 'company': 'Бизнес-метрика', 'experience': 'Без опыта', 'schedule': '', 'employment': 'Полный день', 'skills': ['Знание SQL или python.', 'Знание теории data моделирования, проектирования аналитических хранилищ.', 'Знание основ статистики.', 'Умение визуализировать информацию и презентовать идеи и гипотезы.', 'Системное и структурное мышление.'], 'description': b'<div class="vacancy-info__section" data-v-2d74938e=""><div class="vacancy-info-body vacancy-info__body" data-v-2d74938e="" data-v-a06ab32e=""><div class="vacancy-info-body__description" data-v-a06ab32e="">&#1053;&#1072;&#1096;&#1080; &#1087;&#1088;&#1086;&#1077;&#1082;&#1090;&#1099; &#1082;&#1083;&#1072;&#1089;&#1089;&#1080;&#1092;&#1080;&#1094;&#1080;&#1088;&#1091;&#1102;&#1090;&#1089;&#1103; &#1086;&#1090; &#1085;&#1077;&#1073;&#1086;&#1083;&#1100;&#1096;&#1080;&#1093; &#1080;&#1085;&#1090;&#1077;&#1075;&#1088;&#1072;&#1094;&#1080;&